In [1]:
import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips, get_pixel_growth_and_new_children
from subprocess import call
import open3d as o3d
import sklearn
from Analysis.util import *
from directory import directory, path_code
from Analysis.data_info import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests

In [2]:
directory = "/projects/0/einf914/data/"

exp = get_exp((40,0,37),directory)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


In [3]:
exp.load_compressed_skel()

In [27]:
def get_width_hypha(hypha,t):
    nodes, edges = hypha.get_nodes_within(t)
    widths = [edge.width(t) for edge in edges]
    lengths = [len(edge.pixel_list(t)) for edge in edges]
    weighted = [widths[i]*lengths[i] for i in range(len(widths))]
    return(np.sum(weighted)/np.sum(lengths))

def get_rh_bas(exp,criter):
    select_hyph = get_hyph_infos(exp)
    max_speeds = []
    total_growths = []
    lengths = []
    branch_frequ = []
    hyph_l = []
    RH=[]
    BAS=[]
    widths=[]
    for hyph in exp.hyphaes:
        speeds = [c[2] for c in select_hyph[hyph]]
        ts = [c[0] for c in select_hyph[hyph]]
        tp1s = [c[1] for c in select_hyph[hyph]]
        if len(speeds)>0:
            length = hyph.get_length_um(hyph.ts[-1])
            nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
            max_speed = np.max(speeds)
            total_growth = np.sum([speed * get_time(exp,ts[i],tp1s[i]) for i,speed in enumerate(speeds)])
            if criter(max_speed,length):
                RH.append(hyph)
            else:
                BAS.append(hyph)
            lengths.append(length)
            max_speeds.append(max_speed)
            total_growths.append(total_growth)
            branch_frequ.append((len(nodes)-1)/(length+1))
            hyph_l.append(hyph)
            widths.append(get_width_hypha(hyph,hyph.ts[-1]))
        else:
            BAS.append(hyph)            
    return(RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph)

In [5]:
widths=[(hyph,get_width_hypha(hyph,hyph.ts[-1])) for hyph in exp.hyphaes if len(hyph.ts)>0]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [6]:
width_real =[c for c in widths if c[1]<160]
widths_fin = [c[1] for c in width_real]
lengths = [c[0].get_length_um(c[0].ts[-1]) for c in width_real]

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(widths_fin,lengths)
ax.set_xlabel(r'width ($\mu m $)')
ax.set_ylabel(r'length ($\mu m $)')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'length ($\\mu m $)')

In [28]:
RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(get_exp((40,0,37),directory),criter)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [30]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(widths_sp,max_speeds)
ax.set_xlabel(r'width ($\mu m $)')
ax.set_ylabel(r'max speed ($\mu m.s^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'max speed ($\\mu m.s^{-1}$)')

In [11]:
widths_rh = [get_width_hypha(hyph,hyph.ts[-1]) for hyph in RH]
widths_bas = [get_width_hypha(hyph,hyph.ts[-1]) for hyph in BAS if len(hyph.ts)>0]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [25]:
len(widths_rh)

171

In [26]:
len(RH)

180

In [24]:

widths_rh = [width for width in widths_rh if width<160]
widths_bas = [width for width in widths_bas if width<160]
fig = plt.figure()
bins = np.linspace(0, 160, 30)
ax = fig.add_subplot(111)
ax.hist(widths_rh,bins,alpha=0.5,label='rh',density=True)
ax.hist(widths_bas,bins,alpha=0.5,label='bas',density=True)
ax.set_xlabel(r'width ($\mu m $)')
plt.legend(loc='upper right')
# ax.set_ylabel(r'length ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
hyph = choice(RH)
hyph.ts,hyph.mother,[h.ts for h in hyph.mother]

([11, 12, 13, 14, 15, 16, 17],
 [Hyphae(226,229)],
 [[3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37]])

In [80]:
times=[]
widths_t=[]
for rh in RH:
    for t in rh.ts:
        width = get_width_hypha(rh,t)
        if width<80:
            times.append(t)
            widths_t.append(width)

In [81]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(times,widths_t)
ax.set_xlabel('timestep')
ax.set_ylabel(r'width ($\mu m $)')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'width ($\\mu m $)')

In [84]:
slope, intercept, r_value, p_value, std_err = stats.linregress(times,speeds)

In [85]:
speeds= widths_t
zipped_list = zip(times,speeds)
sorted_zip = sorted(zipped_list)
times_sort,speed_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time since start of imaging ($hour$)')
ax.set_ylabel('speed ($\mu m.hour^{-1})$')
xlim = 36
ax.set_xlim((0,xlim))
ax.set_ylim((0,450))

x= range(0,xlim)

N=30
ax.scatter(times_sort,speed_sort,color='yellow',label='observed speeds')
ax.plot(uniform_filter1d(times_sort, size=N),uniform_filter1d(speed_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hyph.ts,[get_width_hypha(hyph,t) for t in hyph.ts])
ax.set_xlabel('timestep')
ax.set_ylabel(r'width ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'width ($\\mu m $)')

In [42]:
hyph.end.show_source_image(hyph.ts[1],hyph.ts[1])
hyph.root.show_source_image(hyph.ts[1],hyph.ts[1])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
hyph.end.ts(),hyph.root.ts()

([35, 36, 37], [36, 37])

In [43]:
plt.close('all')
exp.plot(hyph.ts,[[hyph.end.label,hyph.root.label]]*len(hyph.ts))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
roots=[]
for t in hyph.end.ts():
    #             print(t,tip)
    if hyph.end.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(4819): 2, Node(5274): 1})